In [20]:
import numpy as np
from scipy.stats import norm
from scipy import stats
import random
import numba as nb
from numba.experimental import jitclass
from numba import types,typed, typeof
from numba import int32, float32    # import the types

In [21]:
class Optimal_Decision_Time_Simulation():
    def __init__(self,**kwargs):
        self.TRIAL_TIME = kwargs.get('TRIAL_TIME',1500)
        self.NUM_TRIALS = kwargs.get('NUM_TRIALS',10000)
        self.AGENT_MOVEMENT_TIME = kwargs.get('AGENT_MOVEMENT_TIME',150)
        # Inputs to draw random decision times (THIS USES TIMING UNCERTAINTY)
        self.interval = kwargs.get('interval',25)
        self.agent_start_mean = kwargs.get('agent_start_mean',900)
        self.agent_end_mean = kwargs.get('agent_end_mean',1550)
        self.player_start_mean = kwargs.get('player_start_mean',900)
        self.player_end_mean = kwargs.get('player_end_mean',1550)
        self.agent_decision_means = np.arange(self.agent_start_mean,self.agent_end_mean,self.interval)
        self.player_decision_means = np.arange(self.player_start_mean,self.player_end_mean,self.interval)
        self.m = len(self.player_decision_means)
        self.n = len(self.agent_decision_means)
        
        
        self.reaction_time_mean = kwargs.get('reaction_time_mean')
        self.movement_time_mean = kwargs.get('movement_time_mean')
        self.timing_uncertainty = kwargs.get('timing_uncertainty')
        self.reaction_uncertainty = kwargs.get('reaction_uncertainty')
        self.movement_uncertainty = kwargs.get('movement_uncertainty')
        self.agent_uncertainty = kwargs.get('agent_uncertainty') 
 
        # Random variables inputs
        self.draw_times()
        print(type(self.reaction_time_mean))
        print(type(self.m))
        print(type(self.agent_decision_means))
        
    def draw_times(self):
        self.player_decision_times = np.zeros((self.m,self.n,self.NUM_TRIALS))
        self.reaction_times = np.zeros((self.m,self.n,self.NUM_TRIALS))
        self.movement_times = np.zeros((self.m,self.n,self.NUM_TRIALS))
        self.agent_decision_times = np.zeros((self.m,self.n,self.NUM_TRIALS))
        for i in range(self.m):
            for j in range(self.n):
                self.player_decision_times[i,j,:] = stats.norm.rvs(self.player_decision_means[i],self.timing_uncertainty,size = self.NUM_TRIALS)
                self.reaction_times[i,j,:] = stats.norm.rvs(self.reaction_time_mean,self.reaction_uncertainty,size=self.NUM_TRIALS)
                self.movement_times[i,j,:] = stats.norm.rvs(self.movement_time_mean,self.movement_uncertainty,size=self.NUM_TRIALS)
                self.agent_decision_times[i,j,:] = stats.norm.rvs(self.agent_decision_means[j],self.agent_uncertainty,size = self.NUM_TRIALS)
    def simulation(self):
        if True: # Initialize arrays
            self.player_wins = np.zeros((self.m, self.n))
            self.player_reaction_wins = np.zeros((self.m, self.n))
            self.player_gamble_wins = np.zeros((self.m, self.n))
            
            self.player_incorrects = np.zeros((self.m, self.n))
            self.player_reaction_incorrects = np.zeros((self.m, self.n))
            self.player_gamble_incorrects = np.zeros((self.m, self.n))
            
            self.player_indecisions = np.zeros((self.m, self.n))
            self.player_reaction_indecisions = np.zeros((self.m, self.n))
            self.player_gamble_indecisions = np.zeros((self.m, self.n))
            
            self.player_decision_array = np.empty((self.m, self.n, self.NUM_TRIALS)) #0 is indecision, 1 is win, -1 is loss
            self.agent_decision_array = np.empty((self.m, self.n, self.NUM_TRIALS))
            
            self.player_reach_times = np.empty((self.m, self.n, self.NUM_TRIALS))
            self.player_leave_target_times = np.empty((self.m, self.n, self.NUM_TRIALS))
            self.agent_reach_times = np.empty((self.m, self.n, self.NUM_TRIALS))
        for i in range(self.m):
            for j in range(self.n):
                for k in range(self.NUM_TRIALS):
                     # For gambles, reach time is decision time + movement time.. this get's overwritten for reaction if statement
                     # LEave target time is player decision time for gambles, again get's changed in first if statement
                    self.player_leave_target_times[i,j,k] = self.player_decision_times[i,j,k]
                    self.player_reach_times[i,j,k] = self.player_decision_times[i,j,k] + self.movement_times[i,j,k]
                    self.agent_reach_times[i,j,k] = self.agent_decision_times[i,j,k] + self.AGENT_MOVEMENT_TIME
                    
                    if ((self.player_decision_times[i,j,k] >= self.agent_decision_times[i,j,k])): # If player selects after agent, they are forced to react
                        self.player_leave_target_times[i,j,k] = self.agent_decision_times[i,j,k] + self.reaction_times[i,j,k] # PLayer can immediately react to the agent
                        self.player_reach_times[i,j,k] = self.player_leave_target_times[i,j,k] + self.movement_times[i,j,k]
                        if self.player_reach_times[i,j,k] > self.TRIAL_TIME:
                            self.player_wins[i,j] += 1
                            self.player_reaction_wins[i,j] += 1
                            self.player_decision_array[i,j,k] = 1
                            self.agent_decision_array[i,j,k] = -1
                            
                        else: # Player doesn't make it
                            self.player_indecisions[i,j]+=1
                            self.player_reaction_indecisions[i,j]+=1
                            self.player_decision_array[i,j,k] = 0
                            self.agent_decision_array[i,j,k] = 1
                            if self.agent_decision_times[i,j,k] + self.AGENT_MOVEMENT_TIME > self.TRIAL_TIME: # Agent doesn't make it either
                                self.agent_decision_array[i,j,k] = 0
                    # NO sensory evidence was used---------------------------------
                    # Both made it
                    elif(self.player_reach_times[i,j,k] < self.TRIAL_TIME and 
                         self.agent_reach_times[i,j,k] < self.TRIAL_TIME):
                        rand = random.random()
                        
                        if rand>=0.5:
                            self.player_wins[i,j] += 1
                            self.player_gamble_wins[i,j] += 1
                            self.player_decision_array[i,j,k] = 1
                            self.agent_decision_array[i,j,k] = -1
                        if rand<0.5:
                            self.player_incorrects[i,j] += 1
                            self.player_gamble_incorrects[i,j] += 1
                            self.player_decision_array[i,j,k] = -1
                            self.agent_decision_array[i,j,k] = 1
                    # Player didn't make it, agent did
                    elif(self.player_reach_times[i,j,k]>=self.TRIAL_TIME and 
                         self.agent_reach_times[i,j,k]<self.TRIAL_TIME):
                        self.player_indecisions[i,j] += 1
                        self.player_gamble_indecisions[i,j]+=1
                        self.player_decision_array[i,j,k] = 0
                        self.agent_decision_array[i,j,k] = 1
                    # Agent didn't make it, player did
                    elif(self.player_reach_times[i,j,k]<self.TRIAL_TIME and 
                         self.agent_reach_times[i,j,k]>=self.TRIAL_TIME):
                        self.player_wins[i,j] += 1
                        self.player_gamble_wins[i,j]+=1
                        self.player_decision_array[i,j,k] = 1
                        self.agent_decision_array[i,j,k] = 0
                    # Both didn't make it
                    elif(self.player_reach_times[i,j,k]>self.TRIAL_TIME and 
                         self.agent_reach_times[i,j,k]>self.TRIAL_TIME):
                        self.player_indecisions[i,j] += 1
                        self.player_gamble_indecisions[i,j]+=1
                        self.player_decision_array[i,j,k] = 0
                        self.agent_decision_array[i,j,k] = 0
                    else:
                        print('ERROR NO IF STATEMENT TRIGGERED')
                            
                        
                    
            
        
        

In [52]:
# Initialize what I want to return
sim = Optimal_Decision_Time_Simulation(reaction_time_mean = 325, movement_time_mean = 90, timing_uncertainty=100, reaction_uncertainty = 30, movement_uncertainty = 30,agent_uncertainty = 50)

<class 'int'>
<class 'int'>
<class 'numpy.ndarray'>


In [57]:
sim.simulation()

In [49]:
np.count_nonzero(np.logical_and(sim.player_decision_array==0,sim.agent_decision_array==0))

121240

In [61]:
np.count_nonzero(np.logical_and(sim.player_decision_times > sim.agent_decision_times,
                                sim.agent_decision_times+sim.movement_times+sim.reaction_times < sim.TRIAL_TIME))

1698363

In [62]:
np.sum(sim.player_reaction_wins)

1681056.0

In [63]:
sim.TRIAL_TIME

1500